In [ ]:
    !pip install PyPDF2
    import os
    import PyPDF2

    def extract_text_from_pdf(pdf_path):
        """Extract text from a single PDF file."""
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in range(len(reader.pages)):
                text += reader.pages[page].extract_text()
        return text

    def extract_text_from_folder(folder_path):
        """Extract text from all PDFs in a folder."""
        extracted_texts = {}
        for filename in os.listdir(folder_path):
            if filename.endswith(".pdf"):
                pdf_path = os.path.join(folder_path, filename)
                text = extract_text_from_pdf(pdf_path)
                extracted_texts[filename] = text
        return extracted_texts

    # Example usage
    folder_path = "/content/drive/MyDrive/chatbot/Pdf" # Replace with the path to your folder containing PDFs
    extracted_texts = extract_text_from_folder(folder_path)

    # Print the extracted text from the first file as a preview
    for filename, text in extracted_texts.items():
        print(f"Extracted text from {filename}:")
        print(text[:500])  # Preview first 500 characters of the text
        print("-" * 50)


Extracted text from Copy of Compendium-2023.pdf:
Preface 
The Right to Information Act, 2005 was promulgated to promote transparency and 
accountability in the working of every public authority. A thorough knowledge of the 
provisions of the Act on the part of the Public Information Officers/First Appellate 
Authorities of the public authorities is key to the successful implementation of the 
Act. 
During the course of evolving of the RTI Act, 2005, Department of Personnel and 
Training (DoPT) has issued clarifications on various provisions of
--------------------------------------------------
Extracted text from Copy of CSJ.A2J.LD. - RTI handbook_.pdf:
R
IGHT
TO
I
NFORMATION
AND
THE
B
UREAUCRACY
:
F
IGHTING
THE
ILLEGALITIES
B
Y
C
ENTRE
FOR
S
OCIAL
J
USTICETable
of
Contents
List
of
Cases
2
Introduction
3
How
to
use
this
Toolkit
4
I.
General
Provisions
5
A.
Important
definitions
under
the
RTI
Act
(with
case
laws)
5
B.
Proactive
Disclosure
Under
Section
4
5
C.
Importance
Of
Section
6
6
D

In [ ]:
!pip install tf-keras
# !pip install sentence-transformers
!pip install --upgrade torch torchvision transformers sentence-transformers

from sentence_transformers import SentenceTransformer

# Load a Hugging Face model for sentence embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_embeddings(texts):
    embeddings = model.encode(texts)  # Converts list of texts into embeddings
    return embeddings

# Example usage
texts = [extracted_texts]  # List of texts (can be split into paragraphs or sentences)
embeddings = generate_embeddings(texts)
print(embeddings[0][:10])  # Preview first 10 elements of the first embedding


[-0.00689321  0.08563083 -0.02562258  0.02191728  0.07077459  0.01442693
  0.00702352 -0.01781001  0.05214915  0.00120309]


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
!pip install faiss-cpu # Install the faiss library, use faiss-cpu for CPU-only version
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import faiss # Now you can import faiss
import numpy as np
import torch

# Example extracted texts
extracted_texts = [
    "Your extracted text goes here.",
    "Add more extracted text here."
]

# ... (rest of your code) ...

# Load DPR Context Encoder model and tokenizer
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Encode texts to generate embeddings
def generate_embeddings(texts):
    inputs = ctx_tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = ctx_encoder(**inputs).pooler_output.numpy()
    return embeddings

embeddings = generate_embeddings(extracted_texts)

# Create FAISS index
def create_faiss_index(embeddings):
    """
    Create a FAISS index for fast similarity search.
    """
    # Convert embeddings to a C-contiguous array if they are not already
    embeddings = np.ascontiguousarray(embeddings)

    d = embeddings.shape[1]  # Dimension of embeddings
    index = faiss.IndexFlatL2(d)  # L2 distance index
    index.add(embeddings)        # Add embeddings to the index
    return index

faiss_index = create_faiss_index(embeddings)

# Query Example (You need a query to retrieve from the FAISS index)
query = "What does the text say about XYZ?"
query_embedding = generate_embeddings([query])[0]

# Search FAISS index
def search_faiss(index, query_embedding, k=5):
    """
    Search the FAISS index and retrieve the top-k closest embeddings.
    """
    distances, indices = index.search(np.array([query_embedding]), k)
    return indices, distances

# Perform search
indices, distances = search_faiss(faiss_index, query_embedding)
print("Top indices:", indices)
print("Distances:", distances)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokeniz

Top indices: [[ 1  0 -1 -1 -1]]
Distances: [[9.1208267e+01 1.2938754e+02 3.4028235e+38 3.4028235e+38 3.4028235e+38]]


In [ ]:
import faiss

# Create a FAISS index for the embeddings
def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 distance (Euclidean)
    # Convert embeddings to a C-contiguous array before adding to the index
    index.add(np.ascontiguousarray(embeddings))
    return index

# Create the index using the embeddings
faiss_index = create_faiss_index(embeddings)
print("FAISS index created!")

FAISS index created!


In [ ]:
def retrieve_similar_texts(query, k=5):
    # Generate embeddings for the query
    query_embedding = generate_embeddings([query])
    distances, indices = faiss_index.search(query_embedding, k)

    # Retrieve the texts corresponding to the indices
    similar_texts = [extracted_texts[idx] for idx in indices[0]]
    return similar_texts, distances[0]

# Example usage
query = "Explain the main points in the document."
results, distances = retrieve_similar_texts(query)
print(f"Top results for query: {query}")
for i, (text, dist) in enumerate(zip(results, distances)):
    print(f"{i + 1}: {text} (distance: {dist})")


Top results for query: Explain the main points in the document.
1: Add more extracted text here. (distance: 98.92103576660156)
2: Your extracted text goes here. (distance: 121.62565612792969)
3: Add more extracted text here. (distance: 3.4028234663852886e+38)
4: Add more extracted text here. (distance: 3.4028234663852886e+38)
5: Add more extracted text here. (distance: 3.4028234663852886e+38)


In [ ]:
!pip install datasets
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load a pre-trained RAG model
rag_model_name = "facebook/rag-token-nq"
tokenizer = RagTokenizer.from_pretrained(rag_model_name)
retriever = RagRetriever.from_pretrained(rag_model_name, index=faiss_index)
rag_model = RagSequenceForGeneration.from_pretrained(rag_model_name)

def generate_response(query):
    # Retrieve relevant context
    similar_texts, _ = retrieve_similar_texts(query)
    context = " ".join(similar_texts)  # Combine retrieved texts

    # Tokenize input with context
    inputs = tokenizer.prepare_seq2seq_batch([query], context=context, return_tensors="pt")

    # Generate response
    outputs = rag_model.generate(**inputs)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return response[0]

# Example chatbot interaction
query = "What is the document about?"
response = generate_response(query)
print(f"Chatbot: {response}")


In [ ]:
!pip install farm-haystack[all]


In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline

# Initialize FAISS Document Store
document_store = FAISSDocumentStore(embedding_dim=768)

# Write extracted texts to the document store
docs = [{"content": text} for text in extracted_texts]
document_store.write_documents(docs)

# Initialize Retriever
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)
document_store.update_embeddings(retriever)

# Initialize Reader (QA Model)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

# Build QA Pipeline
pipe = ExtractiveQAPipeline(reader, retriever)

# Ask a Question
query = "What is the document about?"
results = pipe.run(query=query, params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 1}})
print(results)


Writing Documents: 10000it [00:00, 272346.79it/s]       


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.72s/ Batches]

{'query': 'What is the document about?', 'no_ans_gap': -7.027258634567261, 'answers': [<Answer {'answer': 'extracted text', 'type': 'extractive', 'score': 0.028765497729182243, 'context': 'Your extracted text goes here.', 'offsets_in_document': [{'start': 5, 'end': 19}], 'offsets_in_context': [{'start': 5, 'end': 19}], 'document_ids': ['6a34c4a2b4242c7bea713d7eedf4122e'], 'meta': {'vector_id': '0'}}>], 'documents': [<Document: {'content': 'Your extracted text goes here.', 'content_type': 'text', 'score': 0.6314807711633212, 'meta': {'vector_id': '0'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '6a34c4a2b4242c7bea713d7eedf4122e'}>, <Document: {'content': 'Add more extracted text here.', 'content_type': 'text', 'score': 0.6207479984712065, 'meta': {'vector_id': '1'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '9fc2173174ea77242c325054e3ceaf72'}>], 'root_node': 'Query', 'params': {'Retriever': {'top_k': 5}, 'Reader': {'top_k': 1}}, 'node_id': 'Reader'}


In [ ]:
# Example Query
query = "What details are available in the document?"

# Retrieve Answer
result = pipe.run(query=query, params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 1}})
print("Answer:", result["answers"][0].answer)
print("Context:", result["answers"][0].context)


Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.74s/ Batches]

Answer: extracted text
Context: Add more extracted text here.


In [ ]:
query = "Explain the extracted text."

result = pipe.run(query=query, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 3}})
for idx, answer in enumerate(result["answers"]):
    print(f"Answer {idx + 1}: {answer.answer}")
    print(f"Context: {answer.context}")


Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.77s/ Batches]

Answer 1: here
Context: Your extracted text goes here.
Answer 2: Add more extracted text here.
Context: Add more extracted text here.
